In [ ]:
!pip install transformers spacy networkx pandas
!python -m spacy download en_core_web_sm

In [ ]:
import spacy
import sqlite3
from transformers import pipeline
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
nlp=spacy.load("en_core_web_sm")
relationship_extractor = pipeline("zero-shot-classification")

In [ ]:
def extract_entities(text):
    doc=nlp(text)
    entities=[(ent.text, ent.label_) for ent in doc.ents]
    return entities

In [ ]:
def extract_relationships(text, entities):
    relations=[]
    for entity in entities:
        if "neural network" in entity[0].lower():relations.append((entity[0],"type of","machine learning algorithm"))
        if "ANN" in entity[0] or "artificial neural network" in entity[0]:relations.append((entity[0], "is a type of", "neural network"))
        if "CNN" in entity[0] or "convolutional neural network" in entity[0]:relations.append((entity[0], "is a type of", "neural network"))
        if "RNN" in entity[0] or "recurrent neural network" in entity[0]:relations.append((entity[0], "is a type of", "neural network"))
    if "advantages" in text.lower() or "pros" in text.lower():
        for entity in entities:
            if "ANN" in entity[0] or "artificial neural network" in entity[0]:
                relations.append(("ANN","has advantage","flexibility"))
                relations.append(("ANN","has advantage","complex pattern recognition"))
                relations.append(("ANN","has advantage","generalization from data"))
            if "CNN" in entity[0] or "convolutional neural network" in entity[0]:
                relations.append(("CNN","has advantage","feature extraction from images"))
                relations.append(("CNN","has advantage","quick training"))
                relations.append(("CNN","has advantage","accuracy in image classification"))
            if "RNN" in entity[0] or "recurrent neural network" in entity[0]:
                relations.append(("RNN","has advantage","accuracy in sequential tasks"))
                relations.append(("RNN","has advantage","reduced training time"))
                relations.append(("RNN","has advantage","interpretability"))
    if "disadvantages" in text.lower() or "cons" in text.lower():
        for entity in entities:
            if "ANN" in entity[0] or "artificial neural network" in entity[0]:
                relations.append(("ANN","has disadvantage","large training data requirement"))
                relations.append(("ANN","has disadvantage","overfitting"))
                relations.append(("ANN","has disadvantage","computational expense"))
            if "CNN" in entity[0] or "convolutional neural network" in entity[0]:
                relations.append(("CNN","has disadvantage","hyperparameter tuning complexity"))
                relations.append(("CNN","has disadvantage","computational intensity"))
            if "RNN" in entity[0] or "recurrent neural network" in entity[0]:
                relations.append(("RNN","has disadvantage","large training data requirement"))
                relations.append(("RNN","has disadvantage","slow training"))
                relations.append(("RNN","has disadvantage","interpretability issues"))
    return relations

In [ ]:
def create_db():
    conn=sqlite3.connect('knowledge_graph.db')
    c=conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS entities
                 (id INTEGER PRIMARY KEY, name TEXT, type TEXT)''')
    c.execute('''CREATE TABLE IF NOT EXISTS relationships
                 (id INTEGER PRIMARY KEY, entity1 TEXT, relation TEXT, entity2 TEXT)''')
    c.execute('''CREATE TABLE IF NOT EXISTS chunks
                 (id INTEGER PRIMARY KEY, chunk TEXT)''')
    conn.commit()
    return conn

In [ ]:
def store_entities(conn,entities):
    c=conn.cursor()
    for entity in entities:c.execute("INSERT INTO entities (name, type) VALUES (?, ?)",entity)
    conn.commit()
def store_relationships(conn,relationships):
    c=conn.cursor()
    for relationship in relationships:c.execute("INSERT INTO relationships (entity1, relation, entity2) VALUES (?, ?, ?)",relationship)
    conn.commit()
    
def store_chunks(conn,chunks):
    c=conn.cursor()
    for chunk in chunks:c.execute("INSERT INTO chunks (chunk) VALUES (?)",(chunk,))
    conn.commit()

In [ ]:
def process_text_chunks(text_chunks):
    conn=create_db()
    for chunk in text_chunks:
        entities=extract_entities(chunk)
        relationships =extract_relationships(chunk, entities)
        store_entities(conn,entities)
        store_relationships(conn,relationships)
        store_chunks(conn,[chunk])

In [ ]:
def query_db(conn,question):
    doc = nlp(question)
    entities=[ent.text for ent in doc.ents]
    if not entities:keywords = question.split()
    else:keywords = entities
    c=conn.cursor()
    c.execute("SELECT chunk FROM chunks")
    all_chunks=c.fetchall()
    most_relevant_chunk=None
    max_relevance_score=0
    for chunk in all_chunks:
        relevance_score= relationship_extractor(question,[chunk[0]],multi_label=False)['scores'][0]
        if relevance_score >max_relevance_score:
            max_relevance_score=relevance_score
            most_relevant_chunk= chunk[0]
    return most_relevant_chunk

In [ ]:
def visualize_graph_db(db_path='knowledge_graph.db'):
    conn = sqlite3.connect(db_path)
    c =conn.cursor()
    G = nx.DiGraph()
    c.execute("SELECT name, type FROM entities")
    entities=c.fetchall()
    for entity in entities:G.add_node(entity[0],label=entity[1])
    c.execute("SELECT entity1, relation, entity2 FROM relationships")
    relationships=c.fetchall()
    for rel in relationships:G.add_edge(rel[0],rel[2], label=rel[1])
    conn.close()
    pos = nx.spring_layout(G)
    plt.figure(figsize=(15,10))
    nx.draw_networkx_nodes(G,pos,node_color='skyblue',node_size=2000)
    nx.draw_networkx_labels(G,pos,font_size=12,font_color='black')
    edges=nx.draw_networkx_edges(G,pos,arrowstyle='->',arrowsize=20,edge_color='gray')
    edge_labels=nx.get_edge_attributes(G,'label')
    nx.draw_networkx_edge_labels(G,pos,edge_labels=edge_labels,font_size=10)
    plt.title("Knowledge Graph Visualization")
    plt.show()

In [ ]:
text_chunks = ["There are many different types of neural networks, each with its own advantages and disadvantages. In this blog post, we will compare three of the most popular types: CNN, ANN, and RCNN. We will discuss the pros and cons of each type so that you can decide which is best for your needs.",
    "Neural networks are a type of machine learning algorithm that is used to model complex patterns in data. Neural networks are similar to other machine learning algorithms, but they are composed of a large number of interconnected processing nodes, or neurons, that can learn to recognize patterns of input data.",
    "Neural networks are particularly well suited for tasks that require the recognition of complex patterns, such as image recognition or natural language processing. Neural networks have been used for many years in a variety of fields, but they have become more popular in recent years as advances in computing power and data storage have made them more practical to use.",
    "There are several types of neural network architectures, including the popular convolutional neural network (CNN) and recurrent neural network (RNN). Each type of neural network has its own strengths and weaknesses, so it is important to choose the right architecture for the task at hand."]
process_text_chunks(text_chunks)


In [ ]:
conn=sqlite3.connect('knowledge_graph.db')

In [ ]:
question ="What are ANNs?"
answer = query_db(conn,question)
print("Most relevant chunk:", answer)

In [ ]:
question ="What is Neural Network?"
answer = query_db(conn,question)
print("Most relevant chunk:", answer)

In [ ]:
visualize_graph_db()